## Dados Geográficos: GeoJSON e TopoJSON

Até agora, trabalhamos com *datasets* em JSON e CSV, que correspondem à tabelas de dados compostas de linhas (*records*) e colunas (*fields*). Para que possamos representar regiões geográficas (países, estados, etc.) e trajetórias (roras de voô, linhas de metrô, etc.), precisamos expandir nosso repertório com novas fortamações capazes de trabalhar com formas geométricas diversas.

O [GeoJSON](https://en.wikipedia.org/wiki/GeoJSON) consegue modelar propriedades geográficas utilizando um formato JSON especializado. Uma *propriedade* (`feature`) em um GeoJSON pode incluir informações geográficas &ndash; por exemplo, as coordenadas de `longitude` e `latitude` correspondentes à fronteira de um país &ndash; bem como atributos adicionais dos dados.

Aqui está um objeto `feature` do GeoJSON correspondente à divisa do estado americano do Colorado:

~~~ json
{
  "type": "Feature",
  "id": 8,
  "properties": {"name": "Colorado"},
  "geometry": {
    "type": "Polygon",
    "coordinates": [
      [[-106.32056285448942,40.998675790862656],[-106.19134826714341,40.99813863734313],[-105.27607827344248,40.99813863734313],[-104.9422739227986,40.99813863734313],[-104.05212898774828,41.00136155846029],[-103.57475287338661,41.00189871197981],[-103.38093099236758,41.00189871197981],[-102.65589358559272,41.00189871197981],[-102.62000064466328,41.00189871197981],[-102.052892177978,41.00189871197981],[-102.052892177978,40.74889940428302],[-102.052892177978,40.69733266640851],[-102.052892177978,40.44003613055551],[-102.052892177978,40.3492571857556],[-102.052892177978,40.00333031918079],[-102.04930288388505,39.57414465707943],[-102.04930288388505,39.56823596836465],[-102.0457135897921,39.1331416175485],[-102.0457135897921,39.0466599009048],[-102.0457135897921,38.69751011321283],[-102.0457135897921,38.61478847120581],[-102.0457135897921,38.268861604631],[-102.0457135897921,38.262415762396685],[-102.04212429569915,37.738153927339205],[-102.04212429569915,37.64415206142214],[-102.04212429569915,37.38900413964724],[-102.04212429569915,36.99365914927603],[-103.00046581851544,37.00010499151034],[-103.08660887674611,37.00010499151034],[-104.00905745863294,36.99580776335414],[-105.15404227428235,36.995270609834606],[-105.2222388620483,36.995270609834606],[-105.7175614468747,36.99580776335414],[-106.00829426840322,36.995270609834606],[-106.47490250048605,36.99365914927603],[-107.4224761410235,37.00010499151034],[-107.48349414060355,37.00010499151034],[-108.38081766383978,36.99903068447129],[-109.04483707103458,36.99903068447129],[-109.04483707103458,37.484617466122884],[-109.04124777694163,37.88049961001363],[-109.04124777694163,38.15283644441336],[-109.05919424740635,38.49983761802722],[-109.05201565922046,39.36680339854235],[-109.05201565922046,39.49786885730673],[-109.05201565922046,39.66062637372313],[-109.05201565922046,40.22248895514744],[-109.05201565922046,40.653823231326896],[-109.05201565922046,41.000287251421234],[-107.91779872584989,41.00189871197981],[-107.3183866123281,41.00297301901887],[-106.85895696843116,41.00189871197981],[-106.32056285448942,40.998675790862656]]
    ]
  }
}
~~~

Este `feature` possui um objeto `properties`, que pode incluir uma quantidade arbitrária de _data fields_, e um objeto `geometry`, que, neste caso, contém um único polígono. Este, por sua vez, consiste de coordenadas `[longitude, latitude]` para a fronteira do estado. As coordenadas continuam para o lado direito da tela (arraste para o lado para descobrir...).

Para aprender mais sobre os mínimos detalhes do GeoJSON, veja a [especificação oficial do GeoJSON](http://geojson.org/) ou leia a [cartilha auxiliar de Tom MacWright](https://macwright.org/2015/03/23/geojson-second-bite).

Um contra do GeoJSON como forma de armazenamento é que ele pode conter redundâncias, resultando em arquivos maiores do que o necessário. Por exemplo: Colorado faz divisa com seis outros estados (ou sete, se você contar o vértice compartilhado com o Arizona). Ao invés de utilizarmos listas independetes de coordenadas para cada um destes estados (o que resultaria em diversas interseções), podemos guardar a informação de forma mais compacta salvando cada divisa compartilhada uma única vez, representando a _topologia_ de regiões geográficas. Felizmente, o formato [TopoJSON](https://github.com/topojson/topojson/blob/master/README.md) faz exatamente isso!

Vamos carregar um arquivo TopoJSON dos países do mundo (em uma resolução de 110 metros):

In [ ]:
world = data.world_110m.url
world

'https://vega.github.io/vega-datasets/data/world-110m.json'

In [ ]:
world_topo = data.world_110m()

In [ ]:
world_topo.keys()

dict_keys(['type', 'transform', 'objects', 'arcs'])

In [ ]:
world_topo['type']

'Topology'

In [ ]:
world_topo['objects'].keys()

dict_keys(['land', 'countries'])

_Dê uma olhada no objeto dicionário do TopoJSON (`world_topo`) para ver o seu conteúdo._

Nos dados acima, a propriedade `objects` indica os objetos nomeados que podemos extrair: geometrias de todos os países (`countries`) ou um único polígono representando toda a massa de terra (`land`) do planeta. Ambos podem ser descompactados em arquivos GeoJSON que podemos visualizar.

Como o TopoJSON é um formato especializado, precisamos ensinar o Altair à analisar o formato TopoJSON, indicando qual objeto (elemento nomeado) queremos extrair da topologia. O código a seguir indica que queremos extrair propriedades GeoJSON do _dataset_ `world` para o objeto `countries`:

~~~ js
alt.topo_feature(world, 'countries')
~~~

O método `alt.topo_feature` que chamamos acima expande para o seguinte Vega-Lite JSON:

~~~ json
{
  "values": world,
  "format": {"type": "topojson", "feature": "countries"}
}
~~~

Agora que conseguimos carregar dados geográficos, estamos prontos para criar mapas!

## Marcadores *Geoshape*

Para visualizar dados geográficos, o Altair fornece o tipo de marcador `geoshape`. Para criar um mapa simples, podemos criar um marcador `geoshape` e dar a ele os nossos dados em TopoJSON, que então serão descompactados em propriedades GeoJSON, uma para cada país do mundo:

In [ ]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape()

alt.Chart(...)

No exemplo acima, o Altair aplicar as configurações padrão de cor (azul) e de projeção (`mercator`). Podemos customizar as cores das regiões e a espessura das fronteiras utilizando propriedades padrão de marcadores. Utilizando o método `project`, podemos também modificar a projeção cartográfica:

In [ ]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type='mercator'
)

alt.Chart(...)

Por padrão, o Altair ajusta automaticamente a projeção para que todos os dados caibam dentro do comprimento e altura do gráfico. Podemos também especificar os parâmetros, como `scale` (quantidade de zoom) e `translate` (translação), para customizar as configurações da projeção. A seguir, ajustamos as parâmetros `scale` e `translate` para focar na Europa:

In [ ]:
alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
    fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
).project(
    type='mercator', scale=400, translate=[100, 550]
)

alt.Chart(...)

_Note que a resolução dos dados de 110 metros se torna notável nesta escala. Para visualizar litorais e fronteiras com mais detalhes, precisamos de um arquivo de entrada com geometrias mais refinadas. Ajuste os parâmetros `scale` e `translate` para focar em outras regiões do mapa!_

Até então, o nosso mapa mostra apenas as froneiras entre países. Usando o operador `layer`, podemos combinar múltiplos elementos do mapa. o Altair conta com _geradores de dados_ que podemos utilizar para criar informações para camadas adicionais do mapa:

- O operador *sphere* (`{'sphere': True}`) fornece uma representação em GeoJSON de todas a esféra do planeta. Podemos criar um marcador `geoshape` adicional para preencher o formato da Terra em uma camada de fundo.
- O gerador de *gráticulas* (`{'graticule': ...}`) cria uma propriedade GeoJSON representando uma gratícula: uma grade formada por linhas de latitude e de longitude. Por padrão, a gratícula possui meridianos e paralelos a cada 10° entre ±80° de latitude. Nos polos, temos meridianos a cada 90°. Estas configurações podem ser customizadas usando as propriedades `stepMinor` e `stepMajor`.

Vamos juntar camadas *sphere*, *gratucule* e marcadores de fronteira para criar uma especificação de mapa reutilizável:

In [ ]:
map = alt.layer(
    # use the sphere of the Earth as the base layer
    alt.Chart({'sphere': True}).mark_geoshape(
        fill='#e6f3ff'
    ),
    # add a graticule for geographic reference lines
    alt.Chart({'graticule': True}).mark_geoshape(
        stroke='#ffffff', strokeWidth=1
    ),
    # and then the countries of the world
    alt.Chart(alt.topo_feature(world, 'countries')).mark_geoshape(
        fill='#2a1d0c', stroke='#706545', strokeWidth=0.5
    )
).properties(
    width=600,
    height=400
)

Podemos estender o mapa com uma projeção desejada de visualizar o resultado. A seguir, aplicamos a [Projeção Natural da Terra](https://en.wikipedia.org/wiki/Natural_Earth_projection). A camada _sphere_ fornece o fundo azul-claro e a camada _graticule_, as linhas brancas de referência geográfica.

In [ ]:
map.project(
    type='naturalEarth1', scale=110, translate=[300, 200]
).configure_view(stroke=None)

alt.LayerChart(...)